In [2]:
import tensorflow as tf
import numpy as np
from IPython.display import YouTubeVideo


In [3]:
# to download dataset shards
# curl data.yt8m.org/download.py | shard=1,100 partition=2/frame/train mirror=us python

In [4]:
# Replace with your file path
filenames = ["./train0208.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)


In [5]:
dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [7]:
import tensorflow as tf
import os

filenames = []
for f in os.listdir('./'):
    if f.endswith('.tfrecord'):
        filenames.append(f)
dataset = tf.data.TFRecordDataset(filenames)

feat_rgb = []
labels_list = []

for raw_record in dataset:
    tf_seq_example = tf.train.SequenceExample.FromString(raw_record.numpy())
    n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)
    labels = tf_seq_example.context.feature['labels'].int64_list.value
    id = tf_seq_example.context.feature['id'].bytes_list.value[0]
    rgb_frame = []
    # iterate through frames
    for i in range(n_frames):
        # Decode RGB frames
        rgb_frame.append(tf.io.decode_raw(
                tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0], tf.uint8
            ).numpy().astype(float))
    
    feat_rgb.append(rgb_frame)
    labels_list.append(labels)


In [15]:
total_labels = []
for l in labels_list:
    total_labels.extend(l)
total_labels = list(set(total_labels))
total_labels = max(total_labels) - min(total_labels) + 1
total_labels

3862

In [16]:
max_seq_length = max(len(seq) for seq in feat_rgb)

# Pad sequences
feat_rgb_padded = np.array([np.pad(seq, ((0, max_seq_length - len(seq)), (0, 0)), 'constant') for seq in feat_rgb])

# Function to one-hot encode labels
def one_hot_encode(labels, total_labels):
    one_hot_labels = np.zeros((len(labels), total_labels), dtype=int)
    for i, label_list in enumerate(labels):
        for l in label_list:
            one_hot_labels[i, l] = 1
    return one_hot_labels

labels_one_hot = one_hot_encode(labels_list, total_labels)

feat_rgb_tensor = tf.convert_to_tensor(feat_rgb_padded, dtype=tf.float32)
labels_tensor = tf.convert_to_tensor(labels_one_hot, dtype=tf.int32)

# tf dataset
# dataset = tf.data.Dataset.from_tensor_slices((feat_rgb_tensor, labels_tensor))

# save dataset
save_dir = './saved_dataset'
tf.data.experimental.save(dataset, save_dir)


Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


In [ ]:
len(feat_rgb)

5038

In [ ]:
labels_list

[[8],
 [0, 1, 35, 285, 904],
 [14],
 [31, 40, 47, 57, 386],
 [3, 4],
 [99, 291, 359, 911],
 [28],
 [0, 12],
 [3, 6, 136, 441, 642, 2269],
 [35, 3173],
 [17, 19, 55, 143, 340],
 [1159, 2910],
 [0, 1, 69, 934, 2704],
 [223, 381],
 [0, 1, 51, 77, 212, 2788],
 [5, 16],
 [6, 8],
 [0, 387],
 [0, 607],
 [580, 1099],
 [25],
 [0, 1],
 [2, 160, 231, 273, 491, 544],
 [912, 2434],
 [4, 9, 14],
 [4, 41],
 [15, 499, 723, 946, 1145],
 [0, 27],
 [11, 20, 22, 29, 32, 112, 122, 430, 543, 624, 679],
 [9, 14],
 [15, 536],
 [304],
 [31, 40, 47, 57, 117],
 [24, 158, 279, 1010],
 [106],
 [1, 1441],
 [0, 12, 312],
 [0, 82, 103, 118, 440],
 [11, 528],
 [2, 43, 76, 571],
 [2, 83, 208, 751],
 [0, 1, 42, 487],
 [53, 58, 163, 228, 232],
 [390, 3149],
 [89, 175, 1633],
 [4, 13, 41],
 [2, 30, 55, 408, 515],
 [26, 130, 199, 259, 379],
 [3571],
 [916],
 [61],
 [136, 141, 396],
 [0, 1, 69, 236, 1150, 2017, 2043, 2095],
 [14],
 [26, 259, 379, 735],
 [0, 1, 219],
 [845],
 [2, 30, 1083],
 [3, 511],
 [12],
 [0, 79, 91, 376

In [ ]:
# from feat_rgb and labels_list, create a tensorflow dataset that can be used for training RNN models


In [ ]:
# test feature extractor
from PIL import Image
import os
import numpy
from feature_extractor import YouTube8MFeatureExtractor

# Instantiate extractor. Slow if called first time on your machine, as it
# needs to download 100 MB.
extractor = YouTube8MFeatureExtractor()

image_file = 'cat_pic.jpg'

im = numpy.array(Image.open(image_file))
features = extractor.extract_rgb_frame_features(im)

In [ ]:
features.shape

(1024,)